## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2022/seasontype/2

In [1]:
SET_WEEK = 7
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_7


In [2]:
from scrapers.scrape_2022 import get_2022_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
logit = '../modeling/logit.pkl'
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(logit, 'rb') as file:
    LOGIT = pickle.load(file)

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2022 data

In [4]:
%%time

X = get_2022_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game',
             'offense_downs_Fourth Downs_PCT',
             'offense_passing_AVG',
             'offense_passing_YDS/G',
             'offense_passing_RTG',
             'offense_receiving_AVG',
             'defense_points_per_game',
             'defense_passing_AVG',
             'defense_receiving_AVG',
             'defense_rushing_YDS/G',
             'defense_passing_SYL_per_game',
             'offense_downs_Third Downs_ATT_per_game',
             'defense_downs_Third Downs_ATT_per_game',
             'defense_downs_First Downs_penalty_ratio',
             'offense_passing_TD_per_game',
             'defense_rushing_TD_per_game',
             'defense_pass_TD_per_rush_TD',
             'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 23.2 s


,Team,games_played,season,offense_points_per_game,offense_downs_Fourth Downs_PCT,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,defense_rushing_TD_per_game,defense_pass_TD_per_rush_TD,offense_pass_TD_to_INT
0,Buffalo Bills,6,2022,29.3,60.0,8.1,323.0,106.9,12.3,13.5,...,10.2,76.2,21.000000,12.000000,12.000000,0.114035,2.833333,0.666667,1.250000,4.25
1,Kansas City Chiefs,7,2022,31.9,60.0,8.1,296.0,108.3,12.3,24.6,...,10.5,92.0,20.857143,11.285714,12.857143,0.064935,2.857143,0.428571,5.666667,4.00


In [5]:
print(X.shape)

(32, 21)


In [6]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [7]:
X['games_played'].unique().tolist()

[6, 7]

In [8]:
# X[X['games_played'] == 8]

In [9]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_logit'] = LOGIT.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_logit'] + predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 5
predictions['pred_opt_combo'] = (predictions['pred_logit'] + 7*predictions['pred_catboost']) / 8

predictions.sort_values('pred_opt_combo', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Buffalo Bills,2022,week_7,0.916132,0.376864,0.215874,0.328889,0.413679,0.450287,0.476485
1,Kansas City Chiefs,2022,week_7,0.485175,0.333554,0.256478,0.375437,0.418278,0.373784,0.426640
4,Philadelphia Eagles,2022,week_7,0.164349,0.265670,0.153657,0.267671,0.361131,0.242496,0.336533


In [10]:
predictions.to_csv('weekly_runs_2022/' + week_name + '.csv', index = False)

### Check weekly run

In [11]:
predictions

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Buffalo Bills,2022,week_7,0.916132,0.376864,0.215874,0.328889,0.413679,0.450287,0.476485
1,Kansas City Chiefs,2022,week_7,0.485175,0.333554,0.256478,0.375437,0.418278,0.373784,0.426640
4,Philadelphia Eagles,2022,week_7,0.164349,0.265670,0.153657,0.267671,0.361131,0.242496,0.336533
28,Dallas Cowboys,2022,week_7,0.166125,0.153300,0.099140,0.208270,0.347101,0.194787,0.324479
21,Tampa Bay Buccaneers,2022,week_7,0.129829,0.084486,0.060490,0.180514,0.346210,0.160306,0.319163
10,Cincinnati Bengals,2022,week_7,0.049706,0.233844,0.142309,0.329550,0.352034,0.221489,0.314243
22,Denver Broncos,2022,week_7,0.068345,0.163814,0.102899,0.199308,0.344688,0.175811,0.310145
12,San Francisco 49ers,2022,week_7,0.125055,0.042071,0.062916,0.199325,0.332954,0.152464,0.306966
16,New England Patriots,2022,week_7,0.063732,0.032304,0.048468,0.191287,0.339015,0.134961,0.304605
30,Tennessee Titans,2022,week_7,0.002601,0.004773,0.044900,0.179305,0.346221,0.115560,0.303268
